You can use interrup before or after a node. Eg.
- Asking permission before tool call
- showing the output of the tool call

use the .compile(checkpointer=memory, interrupt_before=[{node}], interrupt_after=[{node}])


In [1]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_tavily import TavilySearch 
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from typing import TypedDict, Annotated, Sequence
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from dotenv import load_dotenv

load_dotenv()

memory = MemorySaver()
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
search_tool = TavilySearch(max_results=2, search_depth='basic')
tools = [search_tool]
tool_model = model.bind_tools(tools=tools)

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

def chat_node(state: AgentState) -> AgentState:
    response = tool_model.invoke(state['messages'])
    return {"messages": [response]}

def tool_router(state: AgentState):
    last_msg = state['messages'][-1]
    if isinstance(last_msg, AIMessage) and getattr(last_msg, "tool_calls", None):
        return "tool_node"
    else:
        return END
    
tool_node = ToolNode(tools=tools)
graph = StateGraph(AgentState)

graph.add_node("chat_node", chat_node)
graph.add_node("tool_node", tool_node)
graph.set_entry_point("chat_node")
graph.add_conditional_edges("chat_node", tool_router)
graph.add_edge("tool_node", "chat_node")

# Using interrupt before tool call
app = graph.compile(checkpointer=memory, interrupt_before=["tool_node"])

In [2]:
config = {"configurable": {"thread_id": 1}}
msg = {"messages": [HumanMessage(content="What is the current weather in Kolkata")]}
events = app.stream(msg, config=config, stream_mode="values")

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the current weather in Kolkata
================================== Ai Message ==================================
Tool Calls:
  tavily_search (a95a469c-bc32-436d-b57e-d3842b837fd7)
 Call ID: a95a469c-bc32-436d-b57e-d3842b837fd7
  Args:
    query: current weather in Kolkata


Mimicking too approval

In [4]:
snapshot = app.get_state(config=config)
print(snapshot.next)
# No need for input since the last saved message contains a tool_call
events = app.stream(None, config=config, stream_mode='values')

for event in events:
    event['messages'][-1].pretty_print()

('tool_node',)
================================== Ai Message ==================================
Tool Calls:
  tavily_search (a95a469c-bc32-436d-b57e-d3842b837fd7)
 Call ID: a95a469c-bc32-436d-b57e-d3842b837fd7
  Args:
    query: current weather in Kolkata
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in Kolkata", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Kolkata", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Kolkata', 'region': 'West Bengal', 'country': 'India', 'lat': 22.5697, 'lon': 88.3697, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1762535471, 'localtime': '2025-11-07 22:41'}, 'current': {'last_updated_epoch': 1762534800, 'last_updated': '2025-11-07 22:30', 'temp_c': 27.0, 'temp_f': 80.6, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 7.4, 